In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


In [ ]:
df = pd.read_csv('data.csv') # df with full ICD-9 codes
df2 = pd.read_csv('data2.csv') # df with first 3 digits of ICD-9 codes

In [ ]:
df_list = []

for idx, row in df.iterrows():
    enrolid = row['ENROLID']
    chf = 0
    htn = 0
    dm = 0
    stroke = 0
    vd = 0
    
    for dx in row[1:].astype(str):
        if pd.isnull(dx):
            continue
        
        if re.search("428\\.0|428\\.1|428\\.2|428\\.20", dx):
            chf = 1
        if re.search("98|99", dx):
            htn = 1
        if re.search("250\\.[0-9]", dx):
            dm = 1
        if re.search("43[0-4]\\.*", dx):
            stroke = 1
        if re.search("410\\.*|411\\.[1,8]|440.0|443.9", dx):
            vd = 1
    
    temp_df = pd.DataFrame({'enrolid': [enrolid], 'chf': [chf], 'htn': [htn], 'dm': [dm], 'stroke': [stroke], 'vd': [vd]})
    df_list.append(temp_df)

# Concatenate all mini DataFrames into the final output DataFrame
output_df = pd.concat(df_list, ignore_index=True)

print(output_df)

In [ ]:
merged_df = pd.merge(output_df, df2, left_on='enrolid', right_on='ENROLID')
chad_vasc_df = merged_df[['chf', 'htn', 'AGE', 'dm', 'stroke', 'vd', 'fem', 'outcome']]
chad_df = merged_df[['chf', 'htn', 'AGE', 'dm', 'stroke', 'outcome']]

In [ ]:
CHADS2_VASC_PTS = [1, 1, 2, 1, 2, 1, 1, 1]

def chad_vasc(row):
    feat = np.array([row['chf'],
                     row['htn'],
                     row['AGE'] >= 75,
                     row['dm'],
                     row['stroke'],
                     row['vd'],
                     65 <= row['AGE'] <= 74,
                     row['fem']], dtype=int)
    return feat.dot(CHADS2_VASC_PTS)

chad_vasc_df['chad_vasc_scores'] = chad_vasc_df.apply(chad_vasc, axis=1)
result_df = chad_vasc_df[['chad_vasc_scores', 'outcome']]
result_df

In [ ]:
CHADS2_PTS = [1, 1, 1, 1, 2]

def chad(row):
    feat = np.array([row['chf'],
                     row['htn'],
                     row['AGE'] >= 75,
                     row['dm'],
                     row['stroke']], dtype=int)
    return feat.dot(CHADS2_PTS)

chad_df['chad_scores'] = chad_df.apply(chad, axis=1).tolist()
result_df_2 = chad_df[['chad_scores', 'outcome']]
result_df_2